# 📊 TEMPLATE REUTILIZÁVEL — EXPLORATORY DATA ANALYSIS (EDA)

## Contexto do Projeto

Descreva aqui:

- Qual problema está sendo resolvido
- Qual decisão de negócio será tomada
- Tipo de problema (classificação, regressão, etc.)
- Variável alvo
- Métrica prioritária

**Exemplo:**
Este projeto tem como objetivo prever churn de clientes, permitindo
ações preventivas de retenção.

## 📚 1. Imports e configurações

In [ ]:
import pandas as pd
import numpy as np
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns

from typing import List

# Adicionar o diretório src ao path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from src.utils import calculate_iv_categorical, calculate_iv_numeric
from src.config import DATA_PATH_RAW

plt.style.use("seaborn-v0_8")
sns.set_palette("Set2")

pd.set_option("display.max_columns", None)

## 📂 2. Carregamento dos Dados

In [ ]:
df = pd.read_csv(DATA_PATH_RAW)

df.head()

## 🔍 3. Visão Geral do Dataset

In [ ]:
print(f"Linhas: {df.shape[0]}")
print(f"Colunas: {df.shape[1]}")

In [ ]:
df.info()

In [ ]:
df.describe()

## 🎯 4. Variável Alvo

In [ ]:
df[TARGET_COL].value_counts(normalize=True)

In [ ]:
sns.countplot(data=df, x=TARGET_COL)
plt.title("Distribuição da Variável Alvo")
plt.show()

### 📌 Checklist

-Dataset balanceado? <br/>
-Accuracy é métrica adequada? <br/>
-Precisa de balanceamento? <br/>

## 🧹 5. Qualidade dos Dados

### 5.1 Valores Ausentes

In [ ]:
missing = df.isna().mean().sort_values(ascending=False)
missing[missing > 0]

### 📌 Checklist:

- Missing é aleatório?  <br/>
- Missing carrega informação?  <br/>
- Remover, imputar ou criar flag?  <br/>

### 5.2 Duplicatas

In [ ]:
if ID_COL in df.columns:
    print("Duplicatas:", df[ID_COL].duplicated().sum())

## 📊 6. Identificação de Tipos de Variáveis

In [ ]:
if NUMERIC_COLS is None:
    NUMERIC_COLS = df.select_dtypes(include=np.number).columns.tolist()

if CATEGORICAL_COLS is None:
    CATEGORICAL_COLS = df.select_dtypes(include="object").columns.tolist()

NUMERIC_COLS, CATEGORICAL_COLS

## 📈 7. Análise Univariada — Numéricas

In [ ]:
for col in NUMERIC_COLS:
    plt.figure(figsize=(6,4))
    sns.histplot(df[col], kde=True)
    plt.title(f"Distribuição de {col}")
    plt.show()

### 📌 Checklist:

- Assimetria?  <br/>
- Outliers?  <br/>
- Transformações necessárias?  <br/>

## 🧩 8. Análise Univariada — Categóricas

In [ ]:
for col in CATEGORICAL_COLS:
    if col not in [ID_COL, TARGET_COL]:
        display(df[col].value_counts(normalize=True))

### 📌 Checklist:

- Cardinalidade alta?  <br/>
- Categorias raras?  <br/>
- Agrupamentos possíveis?  <br/>

## 🔗 9. Análise Bivariada — Numéricas × Target

In [ ]:
for col in NUMERIC_COLS:
    plt.figure(figsize=(6,4))
    sns.boxplot(data=df, x=TARGET_COL, y=col)
    plt.title(f"{col} vs {TARGET_COL}")
    plt.show()

📌 Pergunta-chave:

- O comportamento muda entre as classes?

## 🧩 10. Análise Bivariada — Categóricas × Target

In [ ]:
def churn_rate_by_category(df, col, target):
    return (
        df.groupby(col)[target]
        .apply(lambda x: (x == df[target].unique()[0]).mean())
        .sort_values(ascending=False)
    )

In [ ]:
for col in CATEGORICAL_COLS:
    if col not in [ID_COL, TARGET_COL]:
        display(churn_rate_by_category(df, col, TARGET_COL))

### 📌 Checklist:

- Categrias com risco alto? <br/>
- Variáveis dominantes?  <br/>

## 🔬 11. Análise Multivariada (Opcional)

### 📌 Checklist:

- Target vs (feature1 + feature2) <br/>
- Segmentação por faixas </br>
- Interações importantes </br>

In [ ]:
# Calcular IV para variáveis categóricas e numéricas
iv_results = {}

# Para categóricas
for col in CATEGORICAL_COLS[:2]:  # Primeiras 2 categóricas
    if col not in [ID_COL, TARGET_COL]:
        iv_results[col] = calculate_iv_categorical(df, col, TARGET_COL)

# Para numéricas
for col in NUMERIC_COLS[:2]:  # Primeiras 2 numéricas
    if col != TARGET_COL:
        iv_results[col] = calculate_iv_numeric(df, col, TARGET_COL)

# Exibir resultados ordenados
iv_df = pd.DataFrame(list(iv_results.items()), columns=['Feature', 'IV'])
iv_df = iv_df.sort_values('IV', ascending=False)
print("\n📊 Information Value (IV) das variáveis:")
print("\nInterpretação:")
print("< 0.02: Sem poder preditivo")
print("0.02-0.1: Fraco")
print("0.1-0.3: Médio")
print("0.3-0.5: Forte")
print("> 0.5: Suspeito (overfitting)\n")
display(iv_df)

## 🔥 12. Correlação

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(df[NUMERIC_COLS].corr(), annot=True, cmap="coolwarm")
plt.title("Correlação entre Variáveis Numéricas")
plt.show()

### 📌 Checklist:

- Features redundantes? <br/>
- Multicolinearidade </br>

## 🧠 13. Hipóteses e Insights

Liste hipóteses confirmadas pela EDA:

- 
- 
- 

## 🚀 14. Implicações para Modelagem

### Decisões:
- Encoding necessário
- Features a criar
- Métrica prioritária
- Estratégia de validação